In [1]:
#Importacion de librerias
import re

In [2]:
import numpy as np
import pandas as pd

In [3]:
from pprint import pprint

In [4]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [5]:
from nltk.corpus import stopwords

In [6]:
import spacy

In [7]:
df = pd.read_csv('C:/Users/magutierrez/Documents/data3.csv')

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [10]:
df

,titulo,parrafo
0,7 World Trade Center,"['\n\n\n', ' (', ', ', ', or ', ') refers to t..."
1,"Hebron Church (Intermont, West Virginia)","['\n\n', ' (also historically known as ', ', '..."
2,Harry F. Sinclair House,"['\n\n', 'The ', ' is a mansion at the southea..."
3,Hoysala architecture,"['\n\n', ' is the building style in ', ' devel..."
4,Hoxne Hoard,"['\n\n', '\n', 'The ', ' (', ' ', ')', ' is th..."
...,...,...
1302,Blakeney Chapel,"['\n', '\n\n', ' is a ruined building on the c..."
1303,Belton House,"['\n', ' is a ', ' listed ', ' in the parish o..."
1304,Acra (fortress),"['The ', ' (also spelled ', ', from ', ': ', '..."
1305,Biblioteca Marciana,"['\n', 'The ', ' or ', ' (', ': ', ', but in ..."


In [11]:
df.loc[1,'parrafo']

'[\'\\n\\n\', \' (also historically known as \', \', \', \', and \', \') is a mid-19th-century \', \' church in \', \', \', \', in the U.S. state of \', \'. Hebron Church was founded in 1786 by \', \' in the \', \' Valley, making it the first Lutheran church west of the \', \'. The congregation worshiped in a \', \', which initially served both Lutheran and \', " denominations. Its congregation was originally German-speaking; the church\'s documents and religious services were in German until 1821, when records and sermons transitioned to English.\\n", "The church\'s congregation built the present ", \'-style \', \'-story church building in 1849, when it was renamed Hebron on the Cacapon. The original log church was moved across the road and subsequently used as a \', \', \', \' classroom, and public schoolhouse (attended by future West Virginia governor \', \').\\n\', "To celebrate the congregation\'s 175th anniversary in 1961, Hebron Church constructed a brick community and religious

In [12]:
# Convertir a una lista
data = df.parrafo.values.tolist()
pprint(data[:1])

["['\\n\\n\\n', ' (', ', ', ', or ', ') refers to two buildings that have "
 "existed at the same location within the ', ' in ', ', ', '. The original "
 "structure, part of the ', ', was completed in 1987 and was destroyed in the "
 "', ' in 2001. The current structure opened in May 2006. Both buildings were "
 "developed by ', ', who holds a ', ' for the site from the ']"]


In [13]:
# Eliminar emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
pprint(data[:1])

["['\\n\\n\\n', ' (', ', ', ', or ', ') refers to two buildings that have "
 "existed at the same location within the ', ' in ', ', ', '. The original "
 "structure, part of the ', ', was completed in 1987 and was destroyed in the "
 "', ' in 2001. The current structure opened in May 2006. Both buildings were "
 "developed by ', ', who holds a ', ' for the site from the ']"]


In [14]:
# Eliminar newlines
data = [re.sub(r'\s+', ' ', sent) for sent in data]
pprint(data[:1])

["['\\n\\n\\n', ' (', ', ', ', or ', ') refers to two buildings that have "
 "existed at the same location within the ', ' in ', ', ', '. The original "
 "structure, part of the ', ', was completed in 1987 and was destroyed in the "
 "', ' in 2001. The current structure opened in May 2006. Both buildings were "
 "developed by ', ', who holds a ', ' for the site from the ']"]


In [15]:
# Eliminar comillas
data = [re.sub(r"\'", "", sent) for sent in data]
pprint(data[:1])

['[\\n\\n\\n,  (, , , , or , ) refers to two buildings that have existed at '
 'the same location within the ,  in , , , . The original structure, part of '
 'the , , was completed in 1987 and was destroyed in the ,  in 2001. The '
 'current structure opened in May 2006. Both buildings were developed by , , '
 'who holds a ,  for the site from the ]']


In [16]:
def sentences_to_words(sentences):
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
    return words

data_words = sentences_to_words(data)

print(data_words[:1])

[['or', 'refers', 'to', 'two', 'buildings', 'that', 'have', 'existed', 'at', 'the', 'same', 'location', 'within', 'the', 'in', 'the', 'original', 'structure', 'part', 'of', 'the', 'was', 'completed', 'in', 'and', 'was', 'destroyed', 'in', 'the', 'in', 'the', 'current', 'structure', 'opened', 'in', 'may', 'both', 'buildings', 'were', 'developed', 'by', 'who', 'holds', 'for', 'the', 'site', 'from', 'the']]


In [17]:
# Construimos modelos de bigrams y trigrams

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)

# Aplicamos el conjunto de bigrams/trigrams a nuestros documentos
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [18]:
print(data_words[0])
print(bigram_mod[data_words[0]])
print(trigram_mod[bigram_mod[data_words[0]]])

['or', 'refers', 'to', 'two', 'buildings', 'that', 'have', 'existed', 'at', 'the', 'same', 'location', 'within', 'the', 'in', 'the', 'original', 'structure', 'part', 'of', 'the', 'was', 'completed', 'in', 'and', 'was', 'destroyed', 'in', 'the', 'in', 'the', 'current', 'structure', 'opened', 'in', 'may', 'both', 'buildings', 'were', 'developed', 'by', 'who', 'holds', 'for', 'the', 'site', 'from', 'the']
['or', 'refers', 'to', 'two', 'buildings', 'that', 'have', 'existed', 'at', 'the', 'same', 'location', 'within', 'the', 'in', 'the', 'original_structure', 'part', 'of', 'the', 'was', 'completed', 'in', 'and', 'was', 'destroyed', 'in', 'the', 'in', 'the', 'current', 'structure', 'opened', 'in', 'may', 'both', 'buildings', 'were', 'developed', 'by', 'who', 'holds', 'for', 'the', 'site', 'from', 'the']
['or', 'refers', 'to', 'two', 'buildings', 'that', 'have', 'existed', 'at', 'the', 'same', 'location', 'within', 'the', 'in', 'the', 'original_structure', 'part', 'of', 'the', 'was', 'complet

In [19]:
# python3 -m spacy download en_core_web_trf
# Inicializamos el modelo 'en_core_web_lg' con las componentes de POS únicamente
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [20]:
# Eliminar stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Hacer bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Hacer trigrams
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Lematización basada en el modelo de POS de Spacy
"""
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        print(sent)
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
"""
def lemmatization(texts):
    return texts

In [21]:
# Eliminamos stopwords
data_words_nostops = remove_stopwords(data_words)

In [22]:
data_words_nostops[0]

['refers',
 'two',
 'buildings',
 'existed',
 'location',
 'within',
 'original',
 'structure',
 'part',
 'completed',
 'destroyed',
 'current',
 'structure',
 'opened',
 'may',
 'buildings',
 'developed',
 'holds',
 'site']

In [23]:
# Formamos bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [24]:
data_words_bigrams[:1]

[['refers',
  'two',
  'buildings',
  'existed',
  'location',
  'within',
  'original_structure',
  'part',
  'completed',
  'destroyed',
  'current',
  'structure',
  'opened',
  'may',
  'buildings',
  'developed',
  'holds',
  'site']]

In [25]:
data_words_trigrams = make_trigrams(data_words_bigrams)

In [26]:
data_words_trigrams

[['refers',
  'two',
  'buildings',
  'existed',
  'location',
  'within',
  'original_structure',
  'part',
  'completed',
  'destroyed',
  'current',
  'structure',
  'opened',
  'may',
  'buildings',
  'developed',
  'holds',
  'site'],
 ['also_historically',
  'known',
  'mid_th_century',
  'church',
  'state',
  'hebron_church',
  'founded',
  'valley',
  'making',
  'first',
  'lutheran',
  'church',
  'west',
  'congregation',
  'worshiped',
  'initially_served',
  'lutheran',
  'denominations',
  'congregation',
  'originally',
  'german',
  'speaking',
  'churchs',
  'documents',
  'religious',
  'services',
  'german',
  'records',
  'sermons',
  'transitioned',
  'english',
  'churchs',
  'congregation',
  'built',
  'present',
  'style',
  'story',
  'church',
  'building',
  'renamed',
  'hebron',
  'cacapon',
  'original',
  'log',
  'church',
  'moved',
  'across',
  'road',
  'subsequently',
  'used',
  'classroom',
  'public',
  'schoolhouse',
  'attended',
  'future',

In [27]:
# Lematizamos preservando únicamente noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams)

print(data_lemmatized[:1])

[['refers', 'two', 'buildings', 'existed', 'location', 'within', 'original_structure', 'part', 'completed', 'destroyed', 'current', 'structure', 'opened', 'may', 'buildings', 'developed', 'holds', 'site']]


In [28]:
# Creamos diccionario
id2word = corpora.Dictionary(data_lemmatized)

In [29]:
for key, value in id2word.items():
    print(key, value)

0 buildings
1 completed
2 current
3 destroyed
4 developed
5 existed
6 holds
7 location
8 may
9 opened
10 original_structure
11 part
12 refers
13 site
14 structure
15 two
16 within
17 across
18 also_historically
19 architecturally
20 attended
21 brick
22 building
23 built
24 cacapon
25 celebrate
26 church
27 churchs
28 classroom
29 community
30 compatible
31 congregation
32 congregations
33 constructed
34 continues
35 denominations
36 designed
37 documents
38 education
39 english
40 first
41 founded
42 future
43 german
44 governor
45 hebron
46 hebron_church
47 initially_served
48 known
49 log
50 lutheran
51 making
52 mid_th_century
53 moved
54 october
55 original
56 originally
57 present
58 public
59 records
60 religious
61 renamed
62 road
63 schoolhouse
64 sermons
65 services
66 speaking
67 state
68 story
69 style
70 subsequently
71 th_anniversary
72 transitioned
73 used
74 valley
75 west
76 west_virginia
77 worshiped
78 acquired
79 added
80 america
81 bulk
82 businessmen
83 characteri

1389 instead
1390 late_th_century
1391 led
1392 mentioning
1393 merging
1394 met
1395 plumes
1396 preferred
1397 questioned
1398 recently
1399 refer
1400 referable
1401 relatives
1402 reunion
1403 scientifically
1404 speculations
1405 subfossils
1406 surfaced
1407 tips
1408 travellers
1409 volcanic
1410 widely
1411 wing
1412 awkward
1413 belongs
1414 breed
1415 classified
1416 coast
1417 cup_shaped
1418 displays
1419 distribution
1420 eighteen
1421 extending
1422 face
1423 flight
1424 gray
1425 guarding
1426 hidden
1427 hop
1428 isolated
1429 live
1430 locomotion
1431 mammals
1432 mexico
1433 monogamous
1434 pacific
1435 patchy
1436 prefers
1437 preyed
1438 reptiles
1439 rufous
1440 see_text
1441 southwest
1442 sparrows
1443 spring
1444 streak
1445 streaks
1446 summer
1447 territorial
1448 transverse
1449 twelve
1450 unthreatened
1451 upon
1452 years
1453 additional
1454 bore
1455 debated
1456 exact
1457 green
1458 long_tail
1459 looked
1460 mascarene_islands
1461 mascarene_parrot
1462

2639 hammering
2640 leading
2641 manage
2642 recolonisation
2643 stresses
2644 tendency
2645 wander
2646 woodpeckers
2647 competes
2648 golden
2649 position_within
2650 shares
2651 uncertain
2652 white_eye
2653 cause
2654 extirpation
2655 jamaica
2656 supported
2657 theory
2658 called
2659 goldcrest
2660 irises
2661 kinglet
2662 knight
2663 latters
2664 north_east
2665 pitched
2666 plain
2667 repetition
2668 resembles
2669 separated
2670 available
2671 concealed
2672 eurasia
2673 fattening
2674 interbreed
2675 migration
2676 occupied
2677 particular
2678 predominates
2679 shunned
2680 siberia
2681 treeless
2682 unstreaked
2683 ancient
2684 continue
2685 ducklings
2686 fulvous
2687 hole
2688 leave
2689 lineage
2690 nthe
2691 plentiful
2692 poisoning
2693 reddish_brown
2694 unlined
2695 white_band
2696 access
2697 alcid
2698 auks
2699 bred
2700 clumsy
2701 cultures
2702 developing
2703 easy
2704 egg
2705 era
2706 far_south
2707 faroe
2708 great_auk
2709 great_britain
2710 greenland
2711 

4021 sheepherding
4022 sizable
4023 trot
4024 armour
4025 bulls
4026 closer
4027 cows
4028 java
4029 rhino
4030 rhinoceroses
4031 vietnams
4032 aepycerotini
4033 audiences
4034 bachelor
4035 geography
4036 impala
4037 leaps
4038 strategy
4039 tribe
4040 zoologist
4041 attempts
4042 clutters
4043 echolocation
4044 environmental
4045 forearm
4046 frequency
4047 get
4048 gleaning
4049 hipposideridae
4050 horseshoe_bats
4051 horseshoes
4052 indecisive
4053 leafs
4054 oceania
4055 rhinolophidae
4056 roots
4057 smooth
4058 spiders
4059 swooping
4060 systems
4061 taxa
4062 xa_million_years_ago
4063 allows
4064 bite
4065 blow
4066 brain
4067 cat
4068 deliver
4069 directly
4070 employ
4071 entered
4072 fatal
4073 jaguars
4074 killing
4075 method
4076 pierce
4077 tan
4078 third
4079 transition
4080 humpback
4081 humpbacks
4082 knobbly
4083 lasting
4084 metric
4085 unique
4086 name_comes
4087 abruptly
4088 collector
4089 hammond
4090 natural_history
4091 oryzomyini
4092 rat
4093 rostrum
4094 supp

5388 emerging
5389 enrolled
5390 establishing
5391 firm
5392 graduating
5393 gsd
5394 inspiration
5395 joined
5396 magnate
5397 new_york_city
5398 pei
5399 real
5400 recruited
5401 researching
5402 retired
5403 retirement
5404 retreat
5405 shanghai
5406 sons
5407 transferred
5408 unhappy
5409 villas
5410 cemetery
5411 classes
5412 discovering
5413 egyptology
5414 encouraged
5415 khnum
5416 lecturer
5417 lectures
5418 middle_class
5419 murray
5420 nakht
5421 petries
5422 professor
5423 temple
5424 unwrapping
5425 wage
5426 worker
5427 youth
5428 anglo
5429 arriving
5430 canterbury
5431 cathedral
5432 christchurch
5433 completion
5434 couple
5435 credited
5436 culture
5437 designs
5438 elizabeth
5439 emigrant
5440 emigrated
5441 emily
5442 famed
5443 gothic
5444 heavily_influenced
5445 importing
5446 industrial
5447 instrumental
5448 jeweller
5449 lifelong
5450 manufacturer
5451 medieval
5452 mountfort
5453 newman
5454 perfume
5455 philosophy
5456 pilgrims
5457 plaques
5458 provincial
54

6888 stone_circle
6889 stone_circles_survive
6890 stones_represented
6891 surveyed
6892 unknown_although
6893 western_slope
6894 withypool
6895 withypool_hill
6896 anomalies
6897 baronet
6898 civil
6899 conceived
6900 corporate
6901 dashwood
6902 encapsulates
6903 entertainment
6904 entire
6905 expense
6906 facades
6907 fund
6908 grade
6909 idiosyncratic
6910 landscaped
6911 maintenance
6912 national_trust
6913 park
6914 pleasure
6915 progression
6916 rectangle
6917 resented
6918 satellites
6919 theatrically
6920 upkeep
6921 weddings
6922 wycombe
6923 breaking
6924 cathedrals
6925 constraints
6926 fluted
6927 poetic
6928 replace
6929 romanesque
6930 sees
6931 stonework
6932 survives
6933 truly
6934 unquestionably
6935 west_front
6936 embodies
6937 succeeding
6938 castle
6939 fort
6940 granted
6941 greville
6942 meander
6943 opposite
6944 refortified
6945 ascribed
6946 castle_remained
6947 castles
6948 claverings
6949 counties
6950 descendant
6951 englands
6952 estates
6953 feeble
6954 

8276 supervision
8277 supporters
8278 treasurer
8279 undergraduates
8280 brougham
8281 chosen
8282 confluence
8283 km_south
8284 loyal
8285 vieuxpont
8286 vieuxponts
8287 bramshill
8288 columns
8289 jacobean
8290 listing
8291 matches
8292 panelled
8293 screen
8294 sporting
8295 bramall
8296 bramhall
8297 broom
8298 davenports
8299 expressly
8300 freeholders
8301 grove
8302 grows
8303 lordship
8304 masseys
8305 meanings
8306 nevill
8307 nook
8308 prestigious
8309 reorganisation
8310 residual
8311 secret
8312 smbc
8313 accession
8314 archives
8315 connection
8316 exploring
8317 extension
8318 haringey
8319 rowland
8320 tottenham
8321 tottenhams
8322 aspect
8323 bodiam
8324 ostensibly
8325 permission
8326 situation
8327 watery
8328 buddha
8329 central_dome
8330 perforated
8331 seated
8332 stacked
8333 statues
8334 transcribed
8335 balcony
8336 buckingham
8337 hospitality
8338 monarch
8339 queens
8340 rejoicing
8341 todays
8342 channel
8343 charts
8344 cley
8345 encroaching
8346 glavens
83

In [30]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]]


In [31]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('buildings', 2),
  ('completed', 1),
  ('current', 1),
  ('destroyed', 1),
  ('developed', 1),
  ('existed', 1),
  ('holds', 1),
  ('location', 1),
  ('may', 1),
  ('opened', 1),
  ('original_structure', 1),
  ('part', 1),
  ('refers', 1),
  ('site', 1),
  ('structure', 1),
  ('two', 1),
  ('within', 1)]]

In [32]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=10, 
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    # beta=1,
    per_word_topics=True
)

In [33]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())

[(0,
  '0.019*"excavation" + 0.018*"collection" + 0.016*"others" + '
  '0.014*"manuscripts" + 0.013*"library" + 0.013*"date" + 0.011*"stone" + '
  '0.011*"one" + 0.010*"ad" + 0.009*"buried"'),
 (1,
  '0.027*"city" + 0.019*"ruins" + 0.018*"site" + 0.016*"xa" + 0.015*"acra" + '
  '0.011*"later" + 0.010*"name" + 0.010*"also" + 0.009*"time" + 0.008*"known"'),
 (2,
  '0.009*"became" + 0.008*"time" + 0.008*"work" + 0.008*"first" + '
  '0.007*"style" + 0.007*"architect" + 0.006*"town" + 0.006*"surrounded" + '
  '0.006*"one" + 0.005*"construction"'),
 (3,
  '0.019*"building" + 0.016*"castle" + 0.011*"structure" + 0.011*"church" + '
  '0.010*"built" + 0.010*"public" + 0.007*"castles" + 0.007*"founded" + '
  '0.006*"used" + 0.006*"first"'),
 (4,
  '0.025*"species" + 0.010*"large" + 0.010*"black" + 0.009*"bird" + '
  '0.008*"range" + 0.008*"two" + 0.008*"white" + 0.008*"long" + 0.008*"found" '
  '+ 0.007*"side"'),
 (5,
  '0.002*"dwelling" + 0.001*"super" + 0.001*"parrot" + 0.000*"night" + '
  '0.

In [34]:
lda_model.alpha

array([0.14728343, 0.27183977, 0.2947064 , 0.28037056, 0.21068972,
       0.00744011, 0.22096989, 0.12842071, 0.20009743, 0.3171903 ],
      dtype=float32)

In [35]:
doc_lda = lda_model[corpus]
print(doc_lda)

In [36]:
# Score de coherencia
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3378235387776824


In [37]:
test_texts = ["The manuscripts excavation "]

In [38]:
test_words = sentences_to_words(test_texts)
print(test_words)

[['the', 'manuscripts', 'excavation']]


In [39]:
tests_words_nostops = remove_stopwords(test_words)
print(tests_words_nostops)

[['manuscripts', 'excavation']]


In [40]:
tests_words_bigrams = make_bigrams(tests_words_nostops)
print(tests_words_bigrams)

[['manuscripts', 'excavation']]


In [41]:
tests_words_trigrams = make_trigrams(tests_words_bigrams)
print(tests_words_trigrams)

[['manuscripts', 'excavation']]


In [42]:
tests_words_bow = [id2word.doc2bow(text) for text in tests_words_trigrams]
print(tests_words_bow)

[[(161, 1), (6453, 1)]]


In [44]:
lda_model.get_document_topics(tests_words_bow[0])

[(0, 0.5264191),
 (1, 0.06664413),
 (2, 0.07225011),
 (3, 0.06873554),
 (4, 0.05165261),
 (6, 0.054172892),
 (7, 0.031483572),
 (8, 0.049055807),
 (9, 0.07776225)]